In [1]:
import random
import pickle
from collections import namedtuple 
from sklearn.model_selection import train_test_split

Transition = namedtuple('Transition',['entity', 'property', 'value'])

In [2]:
with open('tuples_v2_with_topics.pkl', 'rb') as f:
    d = pickle.load(f)
    data = d['data']
    topics_map = d['topics_map']

In [3]:
list(data.items())[0]

('199_1___Test_Your_PS4_Controller___Steps',
 [[Transition(entity='controller', property='isConnected', value='True')],
  [Transition(entity='menu', property='isOpened', value='True')],
  [Transition(entity='control panel', property='isOpened', value='True'),
   Transition(entity='run', property='isOpened', value='True')],
  [Transition(entity='view device and printer', property='isOpened', value='True'),
   Transition(entity='hardware and sound', property='isOpened', value='True')],
  [Transition(entity='wireless controller', property='isOpened', value='True'),
   Transition(entity='ps4', property='isConnected', value='True')],
  [Transition(entity='game controller setting', property='isOpened', value='True')],
  [Transition(entity='property', property='isOpened', value='True')]])

In [4]:
data_k, data_v = [], []

for k, v in data.items():
    data_k.append(k)
    data_v.append(v)


In [5]:
ct = 0
for procedure in data_v:
    if len(procedure[0]) > 1: # or len(procedure[1]) > 1:
        #print(el)
        ct+=1
ct

330

In [6]:
train, test = train_test_split(data_v,
                               test_size=0.25,
                               shuffle=True,
                               random_state=42, 
                               stratify=[topics_map[x] for x in data_k],
                              )

In [7]:
len(train), len(test) #, len(train[0])

(932, 311)

In [8]:
ct = 0
for procedure in test:
    if len(procedure) < 2:# or len(procedure[0]) > 1 or len(procedure[1]) > 1:
        ct+=1  
ct

19

In [9]:
ct = 0
for procedure in test:
    if len(procedure[0]) > 1: # or len(procedure[1]) > 1:
        ct+=1
ct

88

In [10]:
ct = 0
for procedure in test:
    if len(procedure)>1 and len(procedure[1]) > 1:
        ct+=1  
ct

47

In [11]:
# STEP 1 -> STEP 2

results = {'correct': 0, 'incorrect':0, 'tot': 0, 'hits': 0}
for procedure in test:
    if len(procedure) < 2 or len(procedure[0]) > 1 or len(procedure[1]) > 1:
        continue
    query = procedure[0][0]
    gold = procedure[1][0]
    hit = False
    done  = False
    for procedure in train:
        if len(procedure) < 2:
            continue
        for i, step in enumerate(procedure[:-1]):
            if query in step:
                hit = True
                preds = procedure[i+1]
                if gold in preds:
                    done= True
                    results['correct'] += 1
                    break
                #else:
                #    hit = True
        if done:
            break
        
    if not done:
        results['incorrect'] += 1
    results['tot'] +=1

    if hit:
        results['hits'] += 1
        

results             

{'correct': 59, 'incorrect': 122, 'tot': 181, 'hits': 129}

In [12]:
# STEP 1 + STEP 2 -> STEP 3

results = {'correct': 0, 'incorrect':0, 'tot': 0, 'hits': 0}
for procedure in test:
    if len(procedure) < 3 or len(procedure[0]) > 1 or len(procedure[1]) > 1 or len(procedure[2]) > 1:
        continue
    query0 = procedure[0][0]
    query1 = procedure[1][0]
    gold = procedure[2][0]
    hit = False
    done  = False
    for procedure in train:
        if len(procedure) < 3:
            continue
        for i, step in enumerate(procedure[1:-1]):
            if query1 in step and query0 in procedure[i-1]:
                hit = True
                preds = procedure[i+1]
                if gold in preds:
                    done= True
                    results['correct'] += 1
                    break
                #else:
                #    hit = True
        if done:
            break
        
    if not done:
        results['incorrect'] += 1
    results['tot'] +=1

    if hit:
        results['hits'] += 1
    
results        

{'correct': 3, 'incorrect': 137, 'tot': 140, 'hits': 8}

In [13]:
# STEP 1 + STEP 2 -> STEP 3

results = {'correct': 0, 'incorrect':0, 'tot': 0, 'hits': 0}
for procedure in test:
    if len(procedure) < 3 or len(procedure[1]) > 1 or len(procedure[2]) > 1:
        continue
    query1 = procedure[1][0]
    gold = procedure[2][0]
    hit = False
    done  = False
    for procedure in train:
        if len(procedure) < 2:
            continue
        for i, step in enumerate(procedure[:-1]):
            if query1 in step:
                hit = True
                preds = procedure[i+1]
                if gold in preds:
                    done= True
                    results['correct'] += 1
                    break
                #else:
                #    hit = True
        if done:
            break
        
    if not done:
        results['incorrect'] += 1
    results['tot'] +=1

    if hit:
        results['hits'] += 1
    
results        

{'correct': 45, 'incorrect': 144, 'tot': 189, 'hits': 122}